In [1]:
from datetime import datetime, time
import numpy as np
import os
import pandas as pd

In [2]:
# Set some global parameters
PATH = '.'
FILE_NAME = 'data/j170520170103.csv'
DATA_STORE = 'store.h5'


## First, define some helper methods

In [3]:
def string_to_date(date_string):
    return datetime.strptime(date_string, '%Y%m%d%H%M%S')

def date_to_string(date):
    return date.strftime('%Y%m%d')

def calculate_stats(df):
    df['theMidPrice'] = (df['theBidPrice1'] + df['theAskPrice1']) / 2.
    df['theVWPrice'] = (df['theBidPrice1'] * df['theBidVolume1'] + df['theAskPrice1'] * df['theAskVolume1']) / (df['theBidVolume1'] + df['theAskVolume1'])
    return df

### Note that here I choose to use hd5f as the data store format instead of the .mat format

HD5F has native support with both pandas and matlab, greater scalability and much higher performance than .mat format. A detailed discussion can be seen [here](https://web.archive.org/web/20151120080623/http://matthewrocklin.com/blog/work/2015/03/16/Fast-Serialization/).

In addition, hd5f supports hierarchical data structure, which enables us to aggregate all the data ranging from all dates into one file instead of dispersing into different .mat file.

In [4]:
def save_data_as_hd5(contract_name, begin_time, end_time, df):
    filename = "{}_{}_{}".format(contract_name, begin_time, end_time)
    store = pd.HDFStore(DATA_STORE)
    store[filename] = df

## Now we are going to load the data

In [5]:
def LoadData(dir_name, file_name, begin_time=None, end_time=None):
    """
    Parameters
    ----------
    dir_name : directory path of the input file (string)
    file_name : file name of the input file (string)
    begin_time : in the format of yyyymmddhhmmss e.g., 20170103090000 (24-hour block)
    end_time: in the format of yyyymmddhhmmss e.g., 20170103090000 
    
    Returns
    -------
    data : a pandas DataFrame object
    """
    column_names = ['theDay','theTime','theMSecond','theBidPrice1','theBidVolume1','theAskPrice1','theAskVolume1','theLastPrice','theVolume',
                   'theAvgPrice','theTurnover']
    path = os.path.join(dir_name, file_name)
    date_parser = lambda x: pd.datetime.strptime(x, '%Y%m%d %H:%M:%S %f')
    usecols = [i for i in range(0, 12) if i != 1]
    raw_data = pd.read_csv(path, names=column_names, usecols=usecols, index_col=False, parse_dates={'theAllTime': ['theDay','theTime','theMSecond']},keep_date_col=True, date_parser=date_parser)
    data = calculate_stats(raw_data)
    
    begin_time = string_to_date(begin_time) if begin_time is not None else data.loc[data.index[0], 'theAllTime']
    end_time = string_to_date(end_time) if end_time is not None else data.loc[data.index[-1], 'theAllTime']
    
    # we do not consider any data before 9:00
    # TODO: Should we consider data after 15:00 ?
    data.set_index('theAllTime', inplace=True)
    date_filter = (data.index.hour >= 9) & (data.index >= begin_time) & (data.index <= end_time)
    data = data.loc[date_filter]
    #save_data_as_hd5(data.loc[data.index[0], 'contract'], date_to_string(begin_time), date_to_string(end_time), data)
    return data
    



In [6]:
df = LoadData(PATH, FILE_NAME)
df.head()

,theDay,theTime,theMSecond,theBidPrice1,theBidVolume1,theAskPrice1,theAskVolume1,theLastPrice,theVolume,theAvgPrice,theTurnover,theMidPrice,theVWPrice
theAllTime,,,,,,,,,,,,,
2017-01-03 09:00:00.486,20170103,09:00:00,486,1519.0,126,1519.0,1,1520.0,4,151907.94,19140400.0,1519.00,1519.000000
2017-01-03 09:00:00.968,20170103,09:00:00,968,1518.5,188,1517.5,1,1518.5,10,151884.04,28554200.0,1518.00,1518.494709
2017-01-03 09:00:01.477,20170103,09:00:01,477,1518.0,236,1515.5,2,1516.5,1,151858.05,35838500.0,1516.75,1517.978992
2017-01-03 09:00:01.973,20170103,09:00:01,973,1517.0,306,1515.0,17,1517.0,4,151822.88,46457800.0,1516.00,1516.894737
2017-01-03 09:00:02.449,20170103,09:00:02,449,1517.5,414,1514.5,1,1516.5,1,151735.51,62818500.0,1516.00,1517.492771


## Now Let's make sure we can recover the data from the storage file

In [7]:
store = pd.HDFStore(DATA_STORE)
store.keys()

[]

In [8]:
df = store['j1705_20170103_20170103']
df.head()

KeyError: 'No object named j1705_20170103_20170103 in the file'